In [1]:
import numpy as np
import pandas as pd
from pandas import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt

import time
import datetime

import json
import os
from dotenv import load_dotenv #pip install python-dotenv

from bs4 import BeautifulSoup
import requests 

from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings
warnings.filterwarnings("ignore")

In [85]:
# comprovar quantes requests he fet i quantes me'n queden

url_search = "https://api.discogs.com/database/search"

params = {
    "type": "release"  # Search for masters
    , "format": "album"           # just search for albums
    , "per_page": 100    # Number of results per page (max 100)
    , "page": 1          # Start with page 1
    , 'country': 'UK'
    , "year": 2016
    , 'genre': 'Rock'
    # , "style": style
}

headers = {"User-Agent": "MyDiscogsApp/1.0", "Authorization": "Discogs token=UwfqmsztxwnfABgQpmhaAsprbUgpOJKGOJSQAqfp"}
response = requests.get(url_search, headers=headers, params=params)

rate_limit = response.headers.get("X-Discogs-Ratelimit")
rate_limit_used = response.headers.get("X-Discogs-Ratelimit-Used")
rate_limit_remaining = response.headers.get("X-Discogs-Ratelimit-Remaining")

print(f"Rate Limit: {rate_limit}")
print(f"Rate Limit Used: {rate_limit_used}")
print(f"Rate Limit Remaining: {rate_limit_remaining}")

Rate Limit: 60
Rate Limit Used: 0
Rate Limit Remaining: 60


### **Proves**: un any en un país

In [86]:
# get all rock_styles in one year in one country

url = "https://api.discogs.com/database/search"
headers = {"User-Agent": 'Arnau', "Authorization": "Discogs token=UwfqmsztxwnfABgQpmhaAsprbUgpOJKGOJSQAqfp"}

genres = ["Blues", "Classical", "Electronic", "Folk, World, & Country", "Funk / Soul", "Hip-Hop", "Jazz", "Latin", "Pop", "Rock", "Stage & Screen"]
rock_styles = ["Pop Rock", "Punk", "Alternative Rock", "Indie Rock", "Hardcore", "Rock & Roll", "Hard Rock", "Psychedelic Rock"
              , "Black Metal", "Heavy Metal", "Folk Rock", "Experimental", "Prog Rock", "Death Metal", "Garage Rock"]

for style in rock_styles:
    # Define parameters for artist search
    params = {
        "per_page": 100    # Number of results per page (max 100)
        , 'type': 'release'
        , 'format': 'album'
        , 'genre': 'Rock'

        # items to change
        , 'country': 'UK'
        , 'year': 2018
        , 'style': style
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    items_number = data['pagination']['items']  # number of items

    print(f"{style}: {items_number}")

Pop Rock: 508
Punk: 950
Alternative Rock: 895
Indie Rock: 937
Hardcore: 255
Rock & Roll: 109
Hard Rock: 240
Psychedelic Rock: 687
Black Metal: 222
Heavy Metal: 212
Folk Rock: 319
Experimental: 479
Prog Rock: 443
Death Metal: 163
Garage Rock: 194


In [6]:
# get one style in one year in one country

url = "https://api.discogs.com/database/search"
headers = {"User-Agent": "MyDiscogsApp/1.0", "Authorization": "Discogs token=UwfqmsztxwnfABgQpmhaAsprbUgpOJKGOJSQAqfp"}

# Define parameters for artist search
params = {
    "per_page": 100    # Number of results per page (max 100)
    , 'type': 'release'
    , 'format': 'album'
    , 'page': 1

    # items to change
    , 'country': 'UK'
    , 'year': 2023
    , 'genre': 'Rock'
    # , 'style': 'Alternative Rock'
}

response = requests.get(url, headers=headers, params=params)
data = response.json()
items_number = response.json()['pagination']['items']  # number of items

print(f"Items: {items_number}")

# response.json()

Items: 5381


### **Rock masters from a specific country**

In [87]:
df_masters_scrapped = pd.read_csv('Datasets/df_masters.csv')
df_masters_scrapped.shape

(13113, 13)

In [8]:
# rock_styles = ["Pop Rock", "Punk", "Alternative Rock", "Indie Rock", "Hardcore", "Rock & Roll",

# Styles scrapped of 2016

In [ ]:
url_search = "https://api.discogs.com/database/search"
headers = {"User-Agent": "MyDiscogsApp", "Authorization": "Discogs token=UwfqmsztxwnfABgQpmhaAsprbUgpOJKGOJSQAqfp"}

df_masters_scrapped = pd.read_csv('Datasets/df_masters.csv')

rock_styles = ["Pop Rock", "Punk", "Alternative Rock", "Indie Rock", "Hardcore", "Rock & Roll", "Hard Rock", "Psychedelic Rock"
              , "Black Metal", "Heavy Metal", "Folk Rock", "Experimental", "Prog Rock", "Death Metal", "Garage Rock"]

# create empty lists
artist_id_list = []
artists_list = []
titles_list = []
years_list = []
release_countries_list = []
album_length_list = []
tracks_list = []
release_type_list = []
genres_list = []
styles_list = []
master_id_new_list = []
artist_profile_list = []
main_release_id_list = []

attempts = 0
scrapped = 0
start_year = 2018

for year in range(start_year, start_year-1, -1):
    print(f'Year: {year}')

    for style in rock_styles:
        print(f'Style: {style}')

        # Define parameters for album search
        params = {
            "type": "release"  # Search for masters
            , "format": "album"           # just search for albums
            , "per_page": 100    # Number of results per page (max 100)
            , "page": 1          # Start with page 1
            , 'country': 'UK'
            , "year": year
            , 'genre': 'Rock'
            , "style": style
        }

        response1 = requests.get(url_search, headers=headers, params=params)
        time.sleep(1)
        data1 = response1.json()

        items = data1['pagination']['items']
        print(f'Items: {items} releases')

        pages = data1['pagination']['pages']+1
        
        for page in range(1, pages):
            print(f'------------ Page: {page} of {pages-1}, {style} of {year} -------------')

            params['page'] = page     # change pages
            response2 = requests.get(url_search, headers=headers, params=params)
            data2 = response2.json()

            for id in range(len(data2['results'])):
                attempts+=1
                time.sleep(1)

                master_id = data2['results'][id]['master_id']

                if master_id == 0:
                    print(f'{attempts} attempts - {scrapped} scrapped - {((page-1)*100)+id+1}/{items} - master_id {master_id}')
                    continue        # I cannot get info 
                elif master_id in master_id_new_list:
                    print(f'{attempts} attempts - {scrapped} scrapped - {((page-1)*100)+id+1}/{items} - master_id {master_id} already in list') 
                    continue        # there are several releases of the same master_release, I just want one (the master_release)
                elif master_id in df_masters_scrapped['master_id'].values:
                    print(f'{attempts} attempts - {scrapped} scrapped - {((page-1)*100)+id+1}/{items} - master_id {master_id} already scrapped') 
                    continue        # I have already scrapped it before
                else:
                    url_master = f"https://api.discogs.com/masters/{master_id}"
                    response3 = requests.get(url_master, headers=headers)
                    time.sleep(1)
                    data3 = response3.json()

                    artist = data3['artists'][0]['name']

                    if artist == 'Various':
                            print(f'{attempts} attempts - {scrapped} scrapped - {((page-1)*100)+id+1}/{items} - master_id {master_id}: Compilation') 
                            continue   # I don't want compilations
                    else:
                    # define the data I want to get
                        title = data3['title']
                        release_year = data3['year']                 
                        genres = data3['genres']
                        styles = data3['styles']

                    # a more complex information is the total duration of the album, I have to sum the durations of each song
                        number_of_songs = len(data3['tracklist'])
                        song_durations = []

                        for song in range(number_of_songs):
                            song_duration = data3['tracklist'][song]['duration']
                            try:
                                minutes = int(song_duration.split(':')[0])
                                seconds = int(song_duration.split(':')[1])
                                song_duration_minutes = minutes + seconds/60
                                song_durations.append(song_duration_minutes)
                            except:
                                pass
                            album_length = round(sum(i for i in song_durations), 2)

                        main_release_id = data3['main_release']
                        artist_id = data3['artists'][0]['id']

                    # release page
                        url = f"https://api.discogs.com/releases/{main_release_id}"
                        time.sleep(1)
                        response4 = requests.get(url, headers=headers)
                        data4 = response4.json()

                    # define the data I want to get
                        release_country = data4['country']
                        release_type = data4['formats'][0]['descriptions']

                    # artist page
                        url = f"https://api.discogs.com/artists/{artist_id}"
                        time.sleep(1)
                        response5 = requests.get(url, headers=headers)
                        data5 = response5.json()

                    # define the data I want to get
                        try:
                            artist_profile = data5['profile']
                        except:
                            artist_profile = np.nan
                            pass

                    # save the info in the lists
                        artist_id_list.append(str(artist_id))
                        artists_list.append(artist)
                        titles_list.append(title)
                        years_list.append(release_year)
                        genres_list.append(genres)
                        styles_list.append(styles)
                        master_id_new_list.append(master_id)
                        album_length_list.append(album_length)
                        tracks_list.append(number_of_songs)
                        release_countries_list.append(release_country)
                        release_type_list.append(release_type)
                        main_release_id_list.append(main_release_id)

                        if artist_profile == '':
                            artist_profile_list.append(np.nan)
                        else:
                            artist_profile_list.append(artist_profile)

                        scrapped+=1

                    # list lengths
                        lists = [artist_id_list, main_release_id_list, master_id_new_list, release_countries_list, artists_list, titles_list, album_length_list
                                 , tracks_list, release_type_list, genres_list, styles_list, artist_profile_list, years_list]

                    # Check if all lists have the same length
                        lengths = [len(lst) for lst in lists]
                        if len(set(lengths)) != 1:
                            print("Lengths are not the same.")
                            break # stop the loop, I won't be able to store the data if I have one value missing

                        print(f'{attempts} attempts - {scrapped} scrapped - {((page-1)*100)+id+1}/{items} - master_id {master_id}: {artist} - {title}')
                        time.sleep(1)

Year: 2018
Style: Pop Rock
Items: 508 releases
------------ Page: 1 of 6, Pop Rock of 2018 -------------
1 attempts - 1 scrapped - 1/508 - master_id 1383263: Diamante (5) - Coming In Hot
2 attempts - 1 scrapped - 2/508 - master_id 0


In [ ]:
# UK 2018

# Pop Rock: 508
# Punk: 950
# Alternative Rock: 895
# Indie Rock: 937
# Hardcore: 255
# Rock & Roll: 109
# Hard Rock: 240
# Psychedelic Rock: 687
# Black Metal: 222
# Heavy Metal: 212
# Folk Rock: 319
# Experimental: 479
# Prog Rock: 443
# Death Metal: 163
# Garage Rock: 194

277+707+138+213+372+489+400+133+193

2922

In [52]:
(112*60)/2922

2.299794661190965

### **Saving the data**

In [55]:
# list lengths
lists = [artist_id_list, master_id_new_list, release_countries_list, artists_list, titles_list, album_length_list, tracks_list , release_type_list, genres_list, styles_list, artist_profile_list, main_release_id_list]

# Check if all lists have the same length
lengths = [len(lst) for lst in lists]
if len(set(lengths)) == 1:
    print(len(master_id_new_list))
else:
    print("Lengths are not the same.")

670


In [56]:
df_masters = pd.DataFrame({'artist_id': artist_id_list
                        , 'master_id': master_id_new_list
                        , 'main_release_id': main_release_id_list
                        , 'release_country': release_countries_list
                        , 'artist': artists_list
                        , 'title': titles_list
                        , 'year': years_list
                        , 'album_length': album_length_list
                        , 'tracks': tracks_list
                        , 'release_type': release_type_list
                        , 'genres': genres_list
                        , 'styles': styles_list
                        , 'artist_profile': artist_profile_list
                        })
df_masters.shape

(670, 13)

In [57]:
print(f'{df_masters.shape[0]} releases\n{df_masters.artist.nunique()} different artists')

670 releases
588 different artists


In [58]:
df_masters.sample(10)

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
599,720072,1085797,9238937,UK,Mithras,On Strange Loops,2016,56.27,12,[Album],[Rock],[Death Metal],Mithras is a technical death metal band that w...
215,1743668,419576,2171827,US,Air (10),Air,1971,42.50,11,"[LP, Album]","[Jazz, Rock]","[Jazz-Rock, Folk, Folk Rock, Prog Rock]",Short-lived jazz-fusion-progressive rock group...
213,4549574,1137113,8795575,UK & Europe,The Cult of Free Love,Love Revolution,2016,42.18,8,"[LP, Album, Limited Edition]","[Electronic, Rock, Folk, World, & Country]","[Experimental, Psychedelic Rock, IDM]",The Cult of Free Love is a musical collective ...
412,1585493,1010000,8603519,UK,Necro Deathmort,The Capsule,2016,0.00,8,"[LP, Album]","[Electronic, Rock]","[Experimental, Krautrock]",Experimental duo from the UK.
390,271950,571765,2662514,UK,David McWilliams,Lord Offaly,1972,44.45,10,"[LP, Album]","[Rock, Folk, World, & Country, Stage & Screen]","[Soft Rock, Folk Rock]","Irish singer/songwriter born on July 4, 1945 i..."
345,4733295,1410927,12407509,UK,Louis Barabbas,Gentle Songs Of Ceaseless Horror,2016,44.37,14,[Album],"[Rock, Folk, World, & Country]","[Folk, Folk Rock, Country]",NaN
80,2965627,1057520,8047699,UK,Gothic Chicken,Lift The Cobweb Veil,2016,42.50,12,"[LP, Album, Limited Edition, Stereo]",[Rock],[Psychedelic Rock],"From Weymouth, UK"
441,3902600,1016479,8597006,UK,Haast's Eagled,II For Mankind,2016,0.00,4,"[LP, Album, Limited Edition]",[Rock],[Doom Metal],Heavy band from South Wales.\r\n\r\nOn Holy Ro...
278,290852,235242,1668313,US,Crowbar (2),Broken Glass,1996,0.00,10,[Album],[Rock],[Sludge Metal],Crowbar is an American heavy / sludge metal ba...
456,2653498,969880,8201722,US,Guerilla Toss,Eraser Stargazer,2016,28.58,8,"[LP, 45 RPM, Album, Limited Edition]",[Rock],"[Experimental, Noise]",Band known for inciting moshing and booty danc...


In [59]:
df_masters.release_country.value_counts().head()

release_country
UK             395
US              94
UK & Europe     35
Europe          33
Germany         14
Name: count, dtype: int64

In [60]:
df_masters.year.value_counts().head()

year
2016    367
2015     21
2012     12
1970     10
1971     10
Name: count, dtype: int64

### **Concat with the previous df**

In [61]:
# import the last df
df_masters_scrapped = pd.read_csv('Datasets/df_masters.csv')
df_masters_scrapped.shape

(12443, 13)

In [62]:
# concat with the df of the albums I just scrapped
df_masters_concat = pd.concat([df_masters_scrapped, df_masters])
df_masters_concat.reset_index(drop=True, inplace=True)
df_masters_concat.shape

(13113, 13)

In [63]:
def check_duplicates(df):
    if df['master_id'].duplicated().sum() == 0:
        print('No duplicates')
    else: 
        duplicates = df['master_id'].duplicated().sum()
        df.drop_duplicates(subset='master_id', inplace=True)
        df.reset_index(drop=True, inplace=True)
        print(f'Found {duplicates} duplicates')
        print(f'Resulting dataset: {df.shape}')

In [64]:
check_duplicates(df_masters_concat)

No duplicates


In [65]:
print(f'{df_masters_concat.shape[0]} releases\n{df_masters_concat.artist.nunique()} different artists')

13113 releases
8189 different artists


### **Exporting to csv**

In [67]:
df_masters_concat.to_csv('Datasets/df_masters.csv', index=False)

In [ ]:
# # import the last df
# df_masters_concat = pd.read_csv('Datasets/df_masters.csv')

### **Statistics**

In [69]:
df_masters_concat.year.value_counts().head(10)

year
2020    1255
2019    1204
2016    1199
2021    1081
2022    1039
2023     975
2024     778
2014     605
2015     462
2018     193
Name: count, dtype: int64

In [70]:
df_masters_concat.release_country.value_counts().head(10)

release_country
UK                 7253
US                 1862
Europe              806
UK & Europe         615
Germany             263
UK, Europe & US     261
UK & US             220
Worldwide           193
Australia           128
Japan               117
Name: count, dtype: int64

In [74]:
df_masters_concat[df_masters_concat['artist']=='King Gizzard And The Lizard Wizard']

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
1381,2532180,830555,6960941,Australia & New Zealand,King Gizzard And The Lizard Wizard,Quarters!,2015,40.67,4,"['LP', 'Album', 'Deluxe Edition', 'Limited Edi...",['Rock'],"['Psychedelic Rock', 'Lo-Fi', 'Jazz-Rock']","Psychedelic rock band from Melbourne, Victoria..."
1396,2532180,912352,7715270,Australia,King Gizzard And The Lizard Wizard,Paper Mâché Dream Balloon,2015,33.77,12,"['LP', 'Album', 'Deluxe Edition', 'Limited Edi...","['Rock', 'Folk, World, & Country']","['Psychedelic Rock', 'Folk Rock']","Psychedelic rock band from Melbourne, Victoria..."
2673,2532180,3459911,30468428,US,King Gizzard And The Lizard Wizard,Nonagon Infinity Live,2024,41.17,8,"['LP', 'Compilation', 'Limited Edition']",['Rock'],"['Garage Rock', 'Psychedelic Rock']","Psychedelic rock band from Melbourne, Victoria..."
4178,2532180,626556,5042612,Australia,King Gizzard And The Lizard Wizard,Float Along - Fill Your Lungs,2013,0.00,8,"['LP', 'Album', 'Limited Edition']",['Rock'],['Psychedelic Rock'],"Psychedelic rock band from Melbourne, Victoria..."
4638,2532180,474693,4050370,Australia,King Gizzard And The Lizard Wizard,12 Bar Bruise,2012,34.30,12,"['LP', 'Album', 'Limited Edition']",['Rock'],"['Garage Rock', 'Psychedelic Rock', 'Surf']","Psychedelic rock band from Melbourne, Victoria..."
5814,2532180,2013943,14648909,Australia,King Gizzard And The Lizard Wizard,Live In Paris '19,2020,92.72,19,"['FLAC', 'Album']",['Rock'],['Psychedelic Rock'],"Psychedelic rock band from Melbourne, Victoria..."
5857,2532180,1268453,11155229,NaN,King Gizzard And The Lizard Wizard,Polygondwanaland,2017,43.77,10,"['FLAC', 'Album']",['Rock'],"['Psychedelic Rock', 'Prog Rock']","Psychedelic rock band from Melbourne, Victoria..."
5914,2532180,1535227,13509733,Australia,King Gizzard And The Lizard Wizard,Fishing For Fishies,2019,41.93,9,"['LP', 'Album', 'Limited Edition']",['Rock'],"['Psychedelic Rock', 'Blues Rock']","Psychedelic rock band from Melbourne, Victoria..."
7577,2532180,2149990,18939442,Australia,King Gizzard And The Lizard Wizard,Live In Sydney '21,2021,89.62,16,['FLAC'],['Rock'],['Psychedelic Rock'],"Psychedelic rock band from Melbourne, Victoria..."
7594,2532180,1850181,16581915,Australia,King Gizzard And The Lizard Wizard,K.G. (Explorations Into Microtonal Tuning Volu...,2020,41.72,10,"['LP', 'Album']",['Rock'],['Psychedelic Rock'],"Psychedelic rock band from Melbourne, Victoria..."


In [75]:
df_masters_concat.artist.value_counts().head(10)

artist
The Fall                              35
Bob Dylan                             28
King Gizzard And The Lizard Wizard    23
Six By Seven                          20
Thee Oh Sees                          18
UFO (5)                               17
Marillion                             17
Porcupine Tree                        17
Schroedinger's Cat                    17
Hawkwind                              16
Name: count, dtype: int64

In [76]:
df_masters_concat.genres.value_counts().head(10)

genres
['Rock']                                     8888
['Electronic', 'Rock']                        923
['Rock', 'Pop']                               879
[Rock]                                        446
['Rock', 'Folk, World, & Country']            416
['Electronic', 'Rock', 'Pop']                 204
['Jazz', 'Rock']                              100
['Rock', 'Blues']                              90
['Rock', 'Pop', 'Folk, World, & Country']      85
[Electronic, Rock]                             68
Name: count, dtype: int64

In [77]:
df_masters_concat.styles.value_counts().head(10)

styles
['Indie Rock']                        769
['Punk']                              491
['Alternative Rock']                  437
[]                                    371
['Black Metal']                       368
['Prog Rock']                         324
['Psychedelic Rock']                  307
['Death Metal']                       222
['Hard Rock']                         220
['Alternative Rock', 'Indie Rock']    219
Name: count, dtype: int64

In [78]:
df_masters_concat[df_masters_concat['release_country']=='Europe'].sort_values('artist')

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
1829,156155,118883,737915,Europe,3 Mustaphas 3,Shopping,1987,44.72,9,"['LP', 'Album']","['Rock', 'Folk, World, & Country']",['Folk Rock'],British band [b]3 Mustaphas 3[/b] did much to ...
4689,3750880,1706746,15028328,Europe,5 Seconds Of Summer,CALM,2020,39.50,12,['Album'],"['Rock', 'Pop']","['Pop Rock', 'Power Pop']",5 Seconds Of Summer (5SOS) is an Australian po...
1326,103279,219113,2075252,Europe,60ft Dolls,The Big 3,1996,41.67,13,['Album'],['Rock'],"['Britpop', 'Indie Rock', 'Hard Rock']","Rock trio from Newport, South Wales, in the st..."
2899,14085388,3566297,31387334,Europe,86TVs,86TVs,2024,0.00,15,"['LP', 'Album', 'Limited Edition']","['Rock', 'Pop']","['Alternative Rock', 'Indie Rock']","Made up of [a499923] Felix and Hugo White, alo..."
10763,8375823,2063863,17972098,Europe,A/lpaca,Make It Better,2021,0.00,9,"['LP', 'Album', 'Limited Edition', 'Stereo']",['Rock'],"['Psychedelic Rock', 'Space Rock', 'Garage Rock']","Mantua, Italy\r\n"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3780,6774153,3038081,26603456,Europe,boygenius,The Record,2023,42.32,12,['Album'],['Rock'],"['Folk Rock', 'Indie Rock']","Phoebe Bridgers, Lucy Dacus and Julien Baker f..."
1088,3971017,725754,6023794,Europe,frnkiero andthe cellabration.,.Stomachaches.,2014,0.00,12,['Album'],['Rock'],[],NaN
3744,12268465,3149436,27982503,Europe,sign crushes motorist,Hurting,2023,30.42,10,"['Album', 'Limited Edition', 'Stereo']",['Rock'],"['Alternative Rock', 'Indie Rock', 'Noisecore']",Add as Sign Crushes Motorist
9400,5609390,1558662,13757895,Europe,whenyoung,Reasons To Dream,2019,41.67,11,['Album'],['Rock'],['Alternative Rock'],"Whenyoung is an Irish indie rock group, origin..."


In [79]:
percentage_album_length_missing = round((df_masters_concat['album_length']==0).sum() / df_masters_concat.shape[0] * 100, 2)
print(f'% missing album_length: {percentage_album_length_missing}%')

% missing album_length: 41.97%


In [80]:
df_masters_concat['artist_profile'].isna().sum()

1838

In [81]:
percentage_artist_profile_missing = round(df_masters_concat['artist_profile'].isna().sum() / df_masters_concat.shape[0] * 100, 2)
print(f'% missing artist_profile: {percentage_artist_profile_missing}%')

% missing artist_profile: 14.02%


In [82]:
df_masters_concat[df_masters_concat['artist_profile']==''] # should be empty

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile


In [19]:
df_masters_concat.loc[10, 'artist_profile']

'English Pop punk band from Southend-on-Sea, Essex, consisting of James Bourne, Matt Willis and Charlie Simpson. Formed in 2000, the band had four UK number-one singles, won two Brit awards and have released four studio albums, selling in excess of five million records.\r\n\r\nIn 2003, Dutch releases of this band were released under the artist name "Bust" to avoid confusion with Dutch band [a493201] who had a hit back then with "We Want More".'

In [84]:
df_masters_concat.groupby(["release_country", 'year']).agg({'master_id': 'count'}).sort_values('master_id', ascending=False).astype(int).head(15)

master_id
release_country year           
UK              2020        741
                2019        717
                2016        669
                2021        602
                2022        577
                2023        551
                2024        428
                2014        308
                2015        267
US              2016        173
                2020        132
                2019        125
                2022        116
                2014        105
                2021         96